## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

### Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

In [2]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    # 모델 학습
    model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 평가
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    cm = confusion_matrix(y_test, y_pred)
    
    # 결과 출력
    print()
    print("--- Model Performance ---")
    print(f"Model Accuracy: {accuracy}")
    print(f"Model F1 Score: {f1}")
    print(f"Model AUC: {auc}")
    
    # 혼동 행렬 출력
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()
    
    return y_pred, y_pred_proba

### Data Load

In [3]:
# 데이터 로드
IVF_train = pd.read_csv('../data/IVF_train_dataset_54.csv')
IVF_test = pd.read_csv('../data/IVF_test_dataset_54.csv')

DI_train = pd.read_csv('../data/DI_train_dataset_54.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_54.csv')

In [4]:
# ID 열을 제외한 특성과 타겟 변수 분리
IVF_X = IVF_train.drop(['임신_성공_여부', 'ID'], axis=1)
IVF_y = IVF_train['임신_성공_여부']

DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [5]:
print(f"IVF_X shape: {IVF_X.shape}")
print(f"IVF_test shape: {IVF_test.drop('ID', axis=1).shape}")

print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

IVF_X shape: (250052, 77)
IVF_test shape: (87891, 77)
DI_X shape: (6289, 31)
DI_test shape: (2176, 31)


### 인코딩 

In [6]:
IVF_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "배란_유도_유형",
    "난자_출처",
    "정자_출처",
    "난자_기증자_나이",
    "정자_기증자_나이",
    "채취_해동_차이",
    "해동_혼합_차이",
    "혼합_이식_차이",
    "이식_해동_차이"
]

In [7]:
DI_categorical_columns = [
    "시술_시기_코드",
    "시술_당시_나이",
    "특정_시술_유형",
    "정자_기증자_나이"
]

In [8]:
# 모든 범주형 변수를 문자열로 변환
IVF_X[IVF_categorical_columns] = IVF_X[IVF_categorical_columns].astype(str)
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
IVF_test[IVF_categorical_columns] = IVF_test[IVF_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
IVF_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

IVF_X[IVF_categorical_columns] = IVF_encoder.fit_transform(IVF_X[IVF_categorical_columns])
DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
IVF_test[IVF_categorical_columns] = IVF_encoder.transform(IVF_test[IVF_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [9]:
# 데이터 분할
IVF_X_train, IVF_X_test, IVF_y_train, IVF_y_test = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier

# 전역 모델 변수 선언
lgb_model = None
xgb_model = None
cat_model = None

# Base 모델 훈련 및 예측 함수 정의
def train_and_predict(X_train, y_train, X_val):
    global lgb_model, xgb_model, cat_model  # 전역 변수 사용 선언

    # 모델 정의
    lgb_model = lgb.LGBMClassifier(verbose=-1)
    xgb_model = xgb.XGBClassifier()
    cat_model = CatBoostClassifier(verbose=0)

    # 모델 훈련
    lgb_model.fit(X_train, y_train)
    xgb_model.fit(X_train, y_train)
    cat_model.fit(X_train, y_train)

    # 예측 확률 생성
    lgb_pred_proba = lgb_model.predict_proba(X_val)[:, 1]
    xgb_pred_proba = xgb_model.predict_proba(X_val)[:, 1]
    cat_pred_proba = cat_model.predict_proba(X_val)[:, 1]

    return lgb_pred_proba, xgb_pred_proba, cat_pred_proba

# IVF 데이터셋에 대한 훈련 및 예측
IVF_X_train, IVF_X_val, IVF_y_train, IVF_y_val = train_test_split(IVF_X, IVF_y, test_size=0.2, random_state=42)

lgbm_ivf_proba, xgb_ivf_proba, cat_ivf_proba = train_and_predict(IVF_X_train, IVF_y_train, IVF_X_val)

# 메타모델 입력 데이터 생성
meta_X_ivf = np.column_stack((lgbm_ivf_proba, xgb_ivf_proba, cat_ivf_proba))

# 메타모델 훈련
meta_model_ivf = LogisticRegression()
meta_model_ivf.fit(meta_X_ivf, IVF_y_val)

# IVF 테스트 데이터에 대한 예측
lgbm_ivf_test_proba = lgb_model.predict_proba(IVF_test.drop('ID', axis=1))[:, 1]
xgb_ivf_test_proba = xgb_model.predict_proba(IVF_test.drop('ID', axis=1))[:, 1]
cat_ivf_test_proba = cat_model.predict_proba(IVF_test.drop('ID', axis=1))[:, 1]

meta_X_ivf_test = np.column_stack((lgbm_ivf_test_proba, xgb_ivf_test_proba, cat_ivf_test_proba))
final_ivf_predictions = meta_model_ivf.predict(meta_X_ivf_test)

# DI 데이터셋에 대한 훈련 및 예측
DI_X_train, DI_X_val, DI_y_train, DI_y_val = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

lgbm_di_proba, xgb_di_proba, cat_di_proba = train_and_predict(DI_X_train, DI_y_train, DI_X_val)

# 메타모델 입력 데이터 생성
meta_X_di = np.column_stack((lgbm_di_proba, xgb_di_proba, cat_di_proba))

# 메타모델 훈련
meta_model_di = LogisticRegression()
meta_model_di.fit(meta_X_di, DI_y_val)

# DI 테스트 데이터에 대한 예측
lgbm_di_test_proba = lgb_model.predict_proba(DI_test.drop('ID', axis=1))[:, 1]
xgb_di_test_proba = xgb_model.predict_proba(DI_test.drop('ID', axis=1))[:, 1]
cat_di_test_proba = cat_model.predict_proba(DI_test.drop('ID', axis=1))[:, 1]

meta_X_di_test = np.column_stack((lgbm_di_test_proba, xgb_di_test_proba, cat_di_test_proba))
final_di_predictions = meta_model_di.predict(meta_X_di_test)

# 결과 출력
print("IVF 최종 예측:", final_ivf_predictions)
print("DI 최종 예측:", final_di_predictions)

IVF 최종 예측: [0 0 0 ... 0 0 0]
DI 최종 예측: [0 0 0 ... 0 0 0]


In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# 모델 정의
lgb_model_ivf = lgb.LGBMClassifier(verbose=-1)
xgb_model_ivf = xgb.XGBClassifier()
cat_model_ivf = CatBoostClassifier(verbose=0)

lgb_model_di = lgb.LGBMClassifier(verbose=-1)
xgb_model_di = xgb.XGBClassifier()
cat_model_di = CatBoostClassifier(verbose=0)

# 전체 데이터를 사용하여 모델 학습 및 예측 함수 정의
def train_and_predict(X_train, y_train, X_test, models):
    # 모델 학습
    for model in models:
        model.fit(X_train, y_train)

    # 예측 확률 생성
    pred_probas = [model.predict_proba(X_test)[:, 1] for model in models]
    return pred_probas

# 공통 특성 찾기
common_features_ivf = list(set(IVF_X.columns) & set(IVF_test.columns))
common_features_di = list(set(DI_X.columns) & set(DI_test.columns))

# ID 열 제거 후 공통 특성을 사용하여 데이터셋 구성
IVF_test_common = IVF_test[common_features_ivf].drop(columns=['ID'], errors='ignore')
DI_test_common = DI_test[common_features_di].drop(columns=['ID'], errors='ignore')

IVF_X_common = IVF_X[common_features_ivf]
DI_X_common = DI_X[common_features_di]

# IVF 데이터에 대한 예측
ivf_models = [lgb_model_ivf, xgb_model_ivf, cat_model_ivf]
lgbm_ivf_proba, xgb_ivf_proba, cat_ivf_proba = train_and_predict(IVF_X_common, IVF_y, IVF_test_common, ivf_models)

# DI 데이터에 대한 예측
di_models = [lgb_model_di, xgb_model_di, cat_model_di]
lgbm_di_proba, xgb_di_proba, cat_di_proba = train_and_predict(DI_X_common, DI_y, DI_test_common, di_models)

# 메타모델 입력 데이터 생성
meta_X_ivf = np.column_stack((lgbm_ivf_proba, xgb_ivf_proba, cat_ivf_proba))
meta_X_di = np.column_stack((lgbm_di_proba, xgb_di_proba, cat_di_proba))

# 메타모델 훈련
meta_model_ivf = LogisticRegression()
meta_model_ivf.fit(meta_X_ivf, IVF_y)  # IVF 데이터에 대한 메타모델 학습

meta_model_di = LogisticRegression()
meta_model_di.fit(meta_X_di, DI_y)  # DI 데이터에 대한 메타모델 학습

# 최종 예측
final_ivf_predictions = meta_model_ivf.predict(meta_X_ivf)
final_di_predictions = meta_model_di.predict(meta_X_di)

# 예측 결과 병합
IVF_test['probability'] = final_ivf_predictions
DI_test['probability'] = final_di_predictions

# 최종 제출 파일 생성
submission = pd.concat([IVF_test[['ID', 'probability']], DI_test[['ID', 'probability']]], axis=0)
submission = submission.sort_values(by='ID')  

# 제출 파일 저장
submission.to_csv('../submission/code54_stacked.csv', index=False, encoding='utf-8')

print("제출 파일이 성공적으로 저장되었습니다.")

ValueError: Found input variables with inconsistent numbers of samples: [87891, 250052]

---

.